<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 4. Evaluate

In [1]:
import glob
import os
import re
import subprocess

import numpy as np
import pandas as pd

from deepcell_tracking.metrics import TrackingMetrics

In [5]:
data_dir = 'data'
models = [
    'Fluo-N2DL-HeLa',
    'Fluo-N2DH-SIM+'
]
seg_dirs = ['gt-seg', 'pred-seg']

pattern = re.compile('\d{3}_GT')

node_match_threshold = 0.6

ctc_software = '../CTC_Evaluation_Software'
operating_system = 'Linux' # or 'Mac' or 'Win'
num_digits = '3'

In [6]:
benchmarks = []

for seg_dir, seg_type in zip(seg_dirs, ['GT', 'Pred']):
    for m in models:
        print(f'------ {m} ------')
        res_subdir = os.path.join(data_dir, seg_dir, m)
        data_ids = [f.split('_')[0] for f in os.listdir(res_subdir) if pattern.fullmatch(f)]

        for data_id in data_ids:
            results = {
                'model': f'CellTrackerGNN {m} - {seg_type}',
                'data_id': data_id
            }
            gt_dir = os.path.join(res_subdir, f'{data_id}_GT/TRA')
            res_dir = os.path.join(res_subdir, f'{data_id}_RES')

            # Deepcell division benchmarking
            try:
                metrics = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir)
                results.update(metrics.stats)
            except ValueError:
                print('Issue with deepcell benchmarking of', data_id)

           # CTC metrics
            for metric, path in [('DET', 'DETMeasure'), ('SEG', 'SEGMeasure'), ('TRA', 'TRAMeasure')]:
                p = subprocess.run([os.path.join(ctc_software, operating_system, path), res_subdir, data_id, num_digits],
                                   stdout=subprocess.PIPE)
                outstring = p.stdout

                try:
                    val = float(outstring.decode('utf-8').split()[-1])
                    results[metric] = val
                except:
                    print('Benchmarking failure', path, m, data_id)
                    print(outstring.decode('utf-8'))

            benchmarks.append(results)

df = pd.DataFrame(benchmarks)
df.to_csv('benchmarks.csv')

------ Fluo-N2DL-HeLa ------
missed node 1_29 division completely
missed node 6_17 division completely
missed node 8_18 division completely
missed node 15_17 division completely
missed node 19_20 division completely
29_17 out degree = 4, daughters mismatch.
missed node 60_34 division completely
missed node 76_7 division completely
missed node 79_9 division completely
83_18 out degree = 1, daughters mismatch.
35_23 out degree = 1, daughters mismatch.
missed node 60_0 division completely
corrected division 60_0 as a frameshift division not an error
missed node 4_26 division completely
missed node 5_53 division completely
missed node 8_42 division completely
missed node 9_38 division completely
12_31 out degree = 3, daughters mismatch.
missed node 15_46 division completely
missed node 18_16 division completely
21_36 out degree = 3, daughters mismatch.
missed node 22_24 division completely
missed node 23_17 division completely
missed node 25_37 division completely
29_32 out degree = 3, dau

In [10]:
df.replace({
    'CellTrackerGNN Fluo-N2DL-HeLa - gt-seg': 'CellTrackerGNN Fluo-N2DL-HeLa - GT',
    'CellTrackerGNN Fluo-N2DH-SIM+ - gt-seg': 'CellTrackerGNN Fluo-N2DH-SIM+ - GT',
    'CellTrackerGNN Fluo-N2DL-HeLa - pred-seg': 'CellTrackerGNN Fluo-N2DL-HeLa - Pred',
    'CellTrackerGNN Fluo-N2DH-SIM+ - pred-seg': 'CellTrackerGNN Fluo-N2DH-SIM+ - Pred'
}).to_csv('/notebooks/benchmarking/CellTrackerGNN/benchmarks.csv')

In [7]:
df['model']

0       CellTrackerGNN Fluo-N2DL-HeLa - gt-seg
1       CellTrackerGNN Fluo-N2DL-HeLa - gt-seg
2       CellTrackerGNN Fluo-N2DL-HeLa - gt-seg
3       CellTrackerGNN Fluo-N2DL-HeLa - gt-seg
4       CellTrackerGNN Fluo-N2DL-HeLa - gt-seg
5       CellTrackerGNN Fluo-N2DL-HeLa - gt-seg
6       CellTrackerGNN Fluo-N2DL-HeLa - gt-seg
7       CellTrackerGNN Fluo-N2DL-HeLa - gt-seg
8       CellTrackerGNN Fluo-N2DL-HeLa - gt-seg
9       CellTrackerGNN Fluo-N2DL-HeLa - gt-seg
10      CellTrackerGNN Fluo-N2DL-HeLa - gt-seg
11      CellTrackerGNN Fluo-N2DL-HeLa - gt-seg
12      CellTrackerGNN Fluo-N2DH-SIM+ - gt-seg
13      CellTrackerGNN Fluo-N2DH-SIM+ - gt-seg
14      CellTrackerGNN Fluo-N2DH-SIM+ - gt-seg
15      CellTrackerGNN Fluo-N2DH-SIM+ - gt-seg
16      CellTrackerGNN Fluo-N2DH-SIM+ - gt-seg
17      CellTrackerGNN Fluo-N2DH-SIM+ - gt-seg
18      CellTrackerGNN Fluo-N2DH-SIM+ - gt-seg
19      CellTrackerGNN Fluo-N2DH-SIM+ - gt-seg
20      CellTrackerGNN Fluo-N2DH-SIM+ - gt-seg
21      CellT